In [ ]:
import matplotlib.pyplot as plt
import pickle
import numpy as np
import scipy

import sys, os

os.chdir(sys.path[0])
os.chdir('..')
from _parameters import *

In [ ]:
# Load TFR
cvsi_avg_f = open(dirs['cvsi'] + '/cvsi_enc1.pkl', 'rb')
cvsi_dat, cvsi_avg = pickle.load(cvsi_avg_f)

cvsi_avg_f.close()

# And time-courses
cvsi_tc_f = open(dirs['cvsi'] + '/cvsi_tc_enc1.pkl', 'rb')
cvsi_tc = pickle.load(cvsi_tc_f)

cvsi_tc_f.close()

In [ ]:
# Load stats

stat_TFR_f = open(dirs['stat'] + '/stat_TFR_enc1.pkl', 'rb')
stat_tc_f = open(dirs['stat'] + '/stat_tc_enc1.pkl', 'rb')

stat_TFR = pickle.load(stat_TFR_f)
stat_TFR_f.close()

stat_tc = pickle.load(stat_tc_f)
stat_tc_f.close()

In [ ]:
# Plot settings
plt.rcdefaults()
plt.rcParams['font.family'] = 'Helvetica Neue'

In [ ]:
def plot_tfr(cond: str, vlim, title, save = False):

    vmin, vmax = vlim

    fig, axes = plt.subplots(figsize = (8,4))

    extent=(-1, 4, 3, 40)

    im = axes.imshow(cvsi_avg[cond], cmap = 'RdBu_r', extent=extent, 
                     origin="lower", aspect="auto", 
                     vmin = vmin, vmax = vmax)
    
    axes.axvline(0, color='grey', linestyle=':')
    axes.axvline(1.5, color='grey', linestyle=':')

    mask = stat_TFR[cond]['mask']

    if len(mask.shape) == 3: # multiple masks
        for m in mask:
            axes.contour(m, colors='black', extent=extent, linewidths=.1, corner_mask=False, antialiased=False)

    else:
        axes.contour(mask, colors='black', extent=extent, linewidths=.1, corner_mask=False, antialiased=False)

    cbar = plt.colorbar(im, ticks = [vmin,0,vmax])
    cbar.ax.tick_params(labelsize = 12)

    axes.set_title(title, size = 14)

    axes.set_xlabel('Time after $\mathregular{1^{st}}$ encoding display (s)', fontsize=14)
    axes.set_ylabel('Frequency (Hz)', fontsize=14)
    axes.tick_params(axis='both', labelsize=12)

    axes.set_xlim(-0.25, 3.5)
    
    if save: 
        fig_fname = dirs['plot'] + '/TFR/' + 'TFR_' + cond.replace('/', '_') + '.svg'
        fig.savefig(fig_fname, format = 'svg', dpi = 300, bbox_inches='tight')

In [ ]:
plot_tfr(cond = 'vis', vlim = [-15,15], title = 'Visual', save = False)
plot_tfr(cond = 'mot', vlim = [-20,20], title = 'Motor', save = False)

In [ ]:
def error_tc(axes, data, time, color):

    avg = np.mean(data, axis=0)

    sd = np.std(data, axis=0)
    se = sd/np.sqrt(len(data))

    axes.fill_between(time, avg-se, avg+se, alpha = 0.2, facecolor = color)
    axes.plot(time, avg, color = color, linestyle = 'solid')


In [ ]:
def plot_tc(cond: list, cols: list, labels: list, title:str, save = False):

    time = cvsi_tc['time']
    stat_loc = [-14, -13]

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize = (8,4), sharex = True)
    
    for i, c in enumerate(cond):
        mask = stat_tc[c]

        error_tc(ax, cvsi_tc[c], time, color = cols[i])
        ax.plot(mask, stat_loc[i] * np.ones(len(mask)), '.', 
                  color = cols[i], label = labels[i])

    ax.axhline(0, color='grey', linestyle=':')
    ax.axvline(1.5, color='grey', linestyle=':')
    ax.axvline(0, color='grey', linestyle=':')

    ax.set_xlim(-0.25, 3.5)
    ax.set_ylim(-15, 15)

    ax.set_title(title, size = 14)

    ax.set_ylabel('Contra vs ipsi (%)', fontsize=14)
    ax.set_xlabel('Time (seconds)', fontsize=14)
    ax.tick_params(axis='both', labelsize=12)

    # legend
    ax.text(0.2, 8, labels[0], color = cols[0], fontsize = 12)
    ax.text(0.2, 6, labels[1], color = cols[1], fontsize = 12)

    if save:
        fig_fname = dirs['plot'] + '/TC/' + 'TC_enc_' + cond[0].split('/')[0] + '.svg'
        fig.savefig(fig_fname, format = 'svg', dpi = 300, bbox_inches='tight')


In [ ]:
colors = ['#96bfe6', '#ff7340']
labels = ['Visual', 'Motor']

plot_tc(cond = ['vis','mot'], 
        cols = colors, labels = labels,
        title = 'Visual and motor', save = True)

In [ ]:
colors = ['#96bfe6', '#ff7340']
labels = ['Same order', 'Reversed order']

plot_tc(cond = ['vis/same', 'vis/rvrs'], 
        cols = colors, labels = labels,
        title = 'Visual encoding (8-12 Hz)', save = True)

plot_tc(cond = ['mot/same', 'mot/rvrs'], 
        cols = colors, labels = labels,
        title = 'Action encoding (13-30 Hz)', save = True)
